In [8]:
#Imports
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [6]:
#Search for target protein
target = new_client.target
target_query = target.search('CHEMBL243')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'Q72874', 'xref_name': None, 'xre...",Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,15.0,False,CHEMBL243,"[{'accession': 'Q72874', 'component_descriptio...",SINGLE PROTEIN,11676


Assign the eigth entry, Influenza A virus Matrix protein M2, to the ***selected_target*** variable and retrieve bioactivity data thats reported as IC<sub>50</sub> values in nM unit.

In [9]:
selected_target = targets.target_chembl_id[0]
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type='IC50')
df = pd.DataFrame.from_dict(res)
df.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32165,[],CHEMBL763424,Inhibitory concentration against HIV-1 Proteas...,B,None,None,BAO_0000190,BAO_0000357,...,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,ug ml-1,UO_0000274,None,30.0
1,None,32166,[],CHEMBL763424,Inhibitory concentration against HIV-1 Proteas...,B,None,None,BAO_0000190,BAO_0000357,...,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,ug ml-1,UO_0000274,None,93.0
2,None,32456,[],CHEMBL769366,In vivo antiviral activity (IC50) against HIV-...,B,None,None,BAO_0000190,BAO_0000218,...,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,nM,UO_0000065,None,42.0
3,None,32459,[],CHEMBL769366,In vivo antiviral activity (IC50) against HIV-...,B,None,None,BAO_0000190,BAO_0000218,...,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,nM,UO_0000065,None,1000.0
4,None,33353,[],CHEMBL696055,Inhibitory activity was determined against HIV...,B,None,None,BAO_0000190,BAO_0000357,...,Human immunodeficiency virus 1,Human immunodeficiency virus type 1 protease,11676,None,None,IC50,nM,UO_0000065,None,4.0


## Datapreprocessing

In [7]:
#Drop missing data
df = df[df.standard_value.notna()]
df = df[df.canonical_smiles.notna()]

#Label compounds as active, inactive or intermediate
bioactivity_class = []
for i in df.standard_value:
    if float(i) >= 100000:
        bioactivity_class.append('inactive')
    elif float(i) <= 1000:
        bioactivity_class.append('active')
    else:
        bioactivity_class.append('intermediate')

#Create dataframe of compund labels and:
#molecule_chembl_id
#canonical_smiles
#standard_value
data_targets = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df2 = df[data_targets]
df2 = pd.concat([df2, pd.Series(bioactivity_class, name='bioactivity_class')], axis=1)
df2.to_csv('bioactivity_data.csv', index=False)
df2

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL9650,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...,30.0,active
1,CHEMBL273396,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...,93.0,active
2,CHEMBL108102,COC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(O)CN(Cc...,42.0,active
3,CHEMBL322033,COC(=O)N(C)C(C(=O)N[C@@H](Cc1ccccc1)[C@H](O)CN...,1000.0,active
4,CHEMBL20143,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccc(OCc...,4.0,active
...,...,...,...,...
3174,CHEMBL376817,COC(=O)[C@H](CC(N)=O)NC(=O)[C@H](CC(C)C)NC(=O)...,5100.0,NaN
3175,CHEMBL224644,CC[C@H](C)[C@H](NC(=O)CCCOc1ccc2ccc(OCCCC(=O)N...,5300.0,NaN
3177,CHEMBL114,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,0.4,NaN
3178,CHEMBL405969,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,290.0,NaN
